In [ ]:
import os    
import pandas as pd
import ollama
import chromadb

In [ ]:
def parse(text):
    abs = text.split("\\\\")[2].strip()
    auth = text.split("\\\\")[1].strip().split("Comment")[0].split(":")[-1]
    return abs,auth


In [ ]:
ids = []
abstracts = []
authors = []

In [ ]:
for folder in os.listdir('cit-HepTh-abstracts'):
    if folder==".DS_Store":
        continue
    for file in os.listdir('cit-HepTh-abstracts/'+folder):
        with open('cit-HepTh-abstracts/'+folder+'/'+file) as f:
            ids.append(file.split(".")[0])
            abstract, author = parse(f.read())
            abstracts.append(abstract)
            authors.append(author)


In [ ]:
print(authors[:100])

In [ ]:
client = chromadb.PersistentClient(path="full_data/")
collection = client.get_or_create_collection(name="authors")

for i, d in enumerate(authors):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[ids[i]],
    embeddings=[embedding],
    documents=[d]
  )
  


In [ ]:
client = chromadb.PersistentClient(path="full_data/")
collection = client.get_or_create_collection(name="docs")

for i, d in enumerate(abstracts):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[ids[i]],
    embeddings=[embedding],
    documents=[d]
  )
  


In [ ]:
len(abstracts)

In [ ]:
client = chromadb.PersistentClient(path="full_data/")
collection = client.get_collection(name="docs")

prompt = """  Effective world-brane actions for solitons of ten-dimensional type IIA and
IIB superstring theory are derived using the formulation of solitons as
Dirichlet branes. The one-brane actions are used to recover predictions of
SL(2,Z) strong-weak coupling duality. The two-brane action, which contains a
hidden eleventh target space coordinate, is shown to be the eleven-dimensional
supermembrane action. It can be thought of as the membrane action of
`M-theory'."""

response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=3,
)
data = results['documents'][0][0]
data